In [1]:
import pandas as pd
import random
import tensorflow as tf
import numpy as np
from tensorflow_federated import python as tff
from collections import OrderedDict
from tensorflow import keras
import os
import random
import linecache
import nest_asyncio
nest_asyncio.apply()
import time
from os import listdir
from os.path import isfile, join

2023-03-13 15:35:59.405393: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
NUM_EPOCHS = 4
# it used to be 4, reduced for debugging
BATCH_SIZE = 40
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10
time_steps =48*3
# *3
interval = 20000 # DO NOT CHANGE!!!!! it was 1000 - this is roughly 80% of 25727 - worked with 20,000
future_steps = 12
split = 0.8

In [3]:
def preprocess(dataset):
    def batch_format_fn(x_d, y_d):
        return OrderedDict(x=x_d, y=y_d)

    return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
    
def create_dataset_fed(files, lower, upper, time_steps=1):
    xs, ys = [], []
    for file in files:
        x_t, y_t = [], []
        data = file[lower:upper]
        if data:
            for i in range(len(data) - time_steps - 1 - future_steps):
                v = data[i:(i + time_steps)] 
                z = data[(i + time_steps):(i + time_steps + future_steps)]
                if check_nulls(z) and check_nulls(v):
                    x_t.append(v)
                    y_t.append(z)
            x_t = np.array(x_t)[:,:,np.newaxis]
            y_t = np.array(y_t)[:,:,np.newaxis]
            xs.append(x_t)
            ys.append(y_t)
    xs = np.array(xs)
    ys = np.array(ys)
    # print("randomrandomrandom")
    # print(xs, "\n this is ys:\n", ys)
    # return [tf.data.Dataset.from_tensor_slices((Xs[x],  np.array(ys[x]))) for x in range(len(Xs))]
    # return [ tf.data.Dataset.from_tensor_slices((Xs[x],  np.array(ys[x]))) for x in range(len(Xs))] - removed brackets, and np.array
    return [tf.data.Dataset.from_tensor_slices((xs[x], ys[x])) for x in range(len(xs))]

def make_federated_data(files, lower, upper):
    data = create_dataset_fed(files, lower, upper,time_steps)
    return [preprocess(x) for x in data if x]

def create_keras_model():
    return tf.keras.models.Sequential([
      keras.layers.LSTM(64, input_shape=(time_steps, 1)),
      keras.layers.Dense(12),
    ])

def model_fn():
    # We _must_ create a new model here, and _not_ capture it from an external
    # scope. TFF will call this within different graph contexts.
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()])

def check_nulls(data):
    if not(all(is_float(ele) for ele in data)):
        return False
    else:
        return True
    
def is_float(element):
    try:
        float(element)
        return True
    except ValueError:
        return False

In [28]:
# clusters = [['1086.txt',
#   '1466.txt',
#   '2521.txt',
#   '2706.txt',
#   '3418.txt',
#   '3766.txt',
#   '3780.txt',
#   '4342.txt',
#   '4690.txt',
#   '4927.txt',
#   '5062.txt',
#   '5445.txt',
#   '5618.txt',
#   '5720.txt',
#   '5844.txt',
#   '5932.txt',
#   '7116.txt'],
#  ['2974.txt'],
#  ['1448.txt',
#   '1632.txt',
#   '1758.txt',
#   '1985.txt',
#   '3757.txt',
#   '4968.txt',
#   '5583.txt',
#   '7136.txt'],
#  ['3676.txt', '5751.txt', '6044.txt', '7353.txt']]
# use min three (have at least one with multple properties)
clusters = [['2822.txt', '3490.txt', '1278.txt', '1652.txt', '2614.txt', '3394.txt', '2471.txt', '2890.txt', '2460.txt', '3421.txt', '1039.txt', '3001.txt', '3375.txt', '3406.txt', '2907.txt', '1360.txt', '2119.txt', '3672.txt', '3869.txt', '2780.txt', '3847.txt', '3338.txt'],
            ['3028.txt'], 
            ['3650.txt', '4063.txt', '3486.txt', '1086.txt', '1733.txt', '1901.txt', '2211.txt', '3182.txt', '1838.txt', '2885.txt', '3769.txt', '4009.txt', '2492.txt', '1170.txt', '2096.txt', '2733.txt', '2041.txt', '3822.txt', '2137.txt', '2494.txt', '1149.txt', '2118.txt', '1516.txt', '3843.txt', '3060.txt', '1716.txt', '2555.txt', '1706.txt', '3850.txt', '2623.txt', '1513.txt'], 
            ['4154.txt'], 
            ['1285.txt', '3669.txt', '3936.txt', '3885.txt'], 
            ['1568.txt']]





In [29]:
# Read files in a dictionary with key = file name, and value = file


C1086 = len(clusters)
PATH = './ExperementData/'

alloc = {}
for c in range(len(clusters)):
    for name in clusters[c]:
        alloc[name] = c
print(alloc)
        
files_in_cluster = []
for c in range(len(clusters) + 1):
    files_in_cluster.append([])
# print(files_in_cluster)

data = []

onlyfiles = [f for f in listdir(PATH) if isfile(join(PATH, f))and f[-4:]==".txt"]
for f in onlyfiles:
    with open (PATH + f,'r') as reader:
        temp = []
        for line in reader:
            if line.strip() == 'Null':
                temp.append('Null')
            else:
                temp.append(float(line.strip()))
                # temp.append(line.strip())
        # print(f, alloc[f])
        data.append(temp)
        files_in_cluster[alloc[f]].append(temp)
        if f == "1086.txt":
            files_in_cluster[C1086].append(temp)

test = [d[int(25727*split):] for d in data]

# print(files_in_cluster)
# print(files_in_cluster[1][1][1])

{'2822.txt': 0, '3490.txt': 0, '1278.txt': 0, '1652.txt': 0, '2614.txt': 0, '3394.txt': 0, '2471.txt': 0, '2890.txt': 0, '2460.txt': 0, '3421.txt': 0, '1039.txt': 0, '3001.txt': 0, '3375.txt': 0, '3406.txt': 0, '2907.txt': 0, '1360.txt': 0, '2119.txt': 0, '3672.txt': 0, '3869.txt': 0, '2780.txt': 0, '3847.txt': 0, '3338.txt': 0, '3028.txt': 1, '3650.txt': 2, '4063.txt': 2, '3486.txt': 2, '1086.txt': 2, '1733.txt': 2, '1901.txt': 2, '2211.txt': 2, '3182.txt': 2, '1838.txt': 2, '2885.txt': 2, '3769.txt': 2, '4009.txt': 2, '2492.txt': 2, '1170.txt': 2, '2096.txt': 2, '2733.txt': 2, '2041.txt': 2, '3822.txt': 2, '2137.txt': 2, '2494.txt': 2, '1149.txt': 2, '2118.txt': 2, '1516.txt': 2, '3843.txt': 2, '3060.txt': 2, '1716.txt': 2, '2555.txt': 2, '1706.txt': 2, '3850.txt': 2, '2623.txt': 2, '1513.txt': 2, '4154.txt': 3, '1285.txt': 4, '3669.txt': 4, '3936.txt': 4, '3885.txt': 4, '1568.txt': 5}


In [30]:
# randomDict = make_federated_data(files_in_cluster[1], 1, 1 + interval)


In [31]:
# Understand interval, time_steps, future_steps, and the NULL values. Then the 20-80 and if there is overlapping train and test data.
# 1086.txt
example_dataset = create_dataset_fed(files_in_cluster[C1086], 1, 500, time_steps)[0]
preprocessed_example_dataset = preprocess(example_dataset)
# process = [tff.learning.build_federated_averaging_process - old code, had to update it
process = [tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: keras.optimizers.Adam(0.001),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)) for _ in range(len(clusters))]
          
state = [process[x].initialize() for x in range (len(process))]
for c in range(len(clusters)):
    max_time = (len(clusters[c])/60)*9000 # CHANGE THIS TO 9000 and 30
    start_time = time.time()
    while time.time() - start_time < max_time:
        location = random.randint(1,int(25727*split)-interval)
        print(location)
        federated_train_data = make_federated_data(files_in_cluster[c], location, location + interval)
        if len(federated_train_data) > 0:
#             state[c], metrics = process[c].next(state[c], federated_train_data) - old code, does not iterate
#             is the below the right thing for state? It does not work for state[c] as this is not iterable
            result = process[c].next(state[c], federated_train_data)
            state[c] = result.state
            metrics = result.metrics
            print('round {:2d}, metrics={}'.format(c, metrics))
#     tensorflow 2.9.0 and federated 0.40.0

384


/var/folders/by/4rt6rrtn24n6r36ktnkzt6q00000gn/T/ipykernel_30489/3823648567.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  xs = np.array(xs)
/var/folders/by/4rt6rrtn24n6r36ktnkzt6q00000gn/T/ipykernel_30489/3823648567.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ys = np.array(ys)


round  0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('root_mean_squared_error', 0.6492432), ('loss', 0.42151675), ('num_examples', 1722412), ('num_batches', 43070)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
260
round  0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('root_mean_squared_error', 0.63243085), ('loss', 0.39996877), ('num_examples', 1721916), ('num_batches', 43057)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
242
round  0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('root_mean_squared_error', 0.6238179), ('loss', 0.3891486), ('num_examples', 1721844), ('num_batches', 43054)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', (

In [32]:
def create_dataset(data, time_steps=1):
    Xs, ys = [], []
    for i in range(len(data) - time_steps-12):
        v = data[i:(i + time_steps)]
        z = data[(i + time_steps):(i + time_steps+12)]
        if check_nulls(z) and check_nulls(v):
            ys.append(z)
            Xs.append(v)
    return np.array(Xs), np.array(ys)

In [33]:
# NEED TO UNDERSTAND WHY THERE ARE NULLS
# fix 80/20 thing?
# why is ther 12 columns in the files
# WHY DO I HAVE A RESULT FOR 60 FILES AND NOT 12 - we train on all files but 80% of the readings and we test on all the houses in experment data but we test on the rest of the 20% of the readings
# why is the training so weird? - trains per cluster and updates the weights in the alloted time so three times per cluster 

# print(len(test))
for i in range(len(test)):
    X_test, y_test = create_dataset(test[i], time_steps)
#     print("HEREEeeeeeeeeeee")
#     print(len(X_test),len(y_test) )
#     print(X_test, "ihbserflijbenlihfb\n", y_test)
    if X_test.size > 0 and y_test.size > 0:
        X_test = X_test[:,:,np.newaxis]
        y_test = y_test[:,:,np.newaxis]
        model_for_inference = create_keras_model()
        weights = state[alloc[onlyfiles[i]]].global_model_weights
    #     weights = process[c].get_model_weights(state[alloc[onlyfiles[i]]]) = guesses
        weights.assign_weights_to(model_for_inference)
    #     state[alloc[onlyfiles[i]]].model.assign_weights_to(model_for_inference) = old code

    #     process.get_model_weights(state[alloc[onlyfiles[i]]]) = what i found online

    #     assigning weights from fed model to new keras model for prediciton
        y_pred = model_for_inference.predict(X_test)
        dataframe = pd.DataFrame(np.squeeze(np.array(y_pred)))
    #     dataframe.to_csv(r"./fed_weight_clust/pred/"+onlyfiles[i][:4]+'.csv')
        dataframe.to_csv(r"./num_clust_six/pred-kmeans/"+onlyfiles[i][:4]+'.csv')
        dataframe = pd.DataFrame(np.squeeze(np.array(y_test)))
    #     dataframe.to_csv(r"./fed_weight_clust/test/"+onlyfiles[i][:4]+'.csv')
        dataframe.to_csv(r"./num_clust_six/test-kmeans/"+onlyfiles[i][:4]+'.csv')
    
    
    
#     created new keras model then assiging weights from other model to this one
#     find new method for fed learning and making new predicitions (.predict or smth)
# .predict
# state is state of fed model that has been trained (model for prediction)

156/156 [==============================] - 3s 18ms/step
